In [7]:
%matplotlib inline

from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import re

telepules = 'Csorna'
num_days = 90

page = requests.get("https://sussfelnap.hu/"+str(num_days)+"-napos-idojaras-elorejelzes/" + telepules)
soup = BeautifulSoup(page.content, 'html.parser')
all_days = soup.find(class_ = "x-scroller")
all_days = all_days.find_all(class_ = "col")


In [9]:
napok = []
napi_min = []
napi_max = []
csapadek_mm = []
csapadek_val = []
szel_km = []
szel_irany = []
for nap in all_days:
    try:
        month = nap.find(class_='withtext').text
    except:
        pass
    try:
        day = ''.join([c for c in nap.select('.title1')[0].text if c in "0123456789."]) + ' ' + nap.select('.title2')[
            0].text
    except:
        day = ''

    n_min = int(nap.select('.min')[0].text.strip())
    n_max = int(nap.select('.max')[0].text.strip())
    try:
        wind = nap.select('.massagealertwind')[0].text.replace('km/h', ' ').split()
    except:
        wind = ['0', '']
    try:
        rain = nap.find('div', re.compile('alertalertpre*')).text.replace('mm', ' ').replace('%', '').split()
    except:
        rain = ['0', '0']

    napok.append(month + ' ' + day)
    napi_min.append(n_min)
    napi_max.append(n_max)
    szel_km.append(int(wind[0]))
    try:
        szel_irany.append(wind[1])
    except:
        szel_irany.append('- ? -')
    
    csapadek_mm.append(int(rain[0]))
    csapadek_val.append(int(rain[1]))

ido = pd.DataFrame({'datum': napok,
                    'minimum': napi_min,
                    'maximum': napi_max,
                    'csap_mm': csapadek_mm,
                    'csap_val': csapadek_val,
                    'szel_km': szel_km,
                    'szel_ir': szel_irany})
max_homerseklet = max(napi_max)
min_homerseklet = min(napi_min)
ido['atlag'] = (ido['minimum'] + ido['maximum']) / 2

In [10]:
print(f'Min.átlag: {ido.minimum.mean():.2f} ℃ , Max.átlag: {ido.maximum.mean():.2f} ℃, Átlag.átlag: {ido.atlag.mean():.2f} ℃\n')
print(f'Abszolút minimum: {ido.minimum.min():>3} ℃\nAbszolút maximum: {ido.maximum.max():>3} ℃')
print(f'Szél maximum: {ido.szel_km.max():>7} km/h')
print(f'Várható csapadék: {ido.csap_mm.sum():>3} mm Valószínűsége: {ido.csap_val.mean():.2f} %')
ido.head()

Min.átlag: -2.41 ℃ , Max.átlag: 8.32 ℃, Átlag.átlag: 2.96 ℃

Abszolút minimum:  -8 ℃
Abszolút maximum:  15 ℃
Szél maximum:      21 km/h
Várható csapadék: 225 mm Valószínűsége: 21.41 %


,datum,minimum,maximum,csap_mm,csap_val,szel_km,szel_ir,atlag
0,November 5. K,7,15,19,65,0,,11.0
1,November 6. Sz,7,13,8,42,0,,10.0
2,November 7. Cs,5,12,3,45,0,,8.5
3,November 8. P,8,14,3,40,21,DK,11.0
4,November 9. Sz,1,12,4,50,0,,6.5


In [ ]:
y_pos = range(num_days)
bar_width = .5

plt.figure(figsize=(22,10))

plt.bar(y_pos, ido['csap_mm'], bar_width, color = 'green', alpha = .5)
plt.bar(y_pos, ido['szel_km']/10, bar_width, color = 'orange', alpha = .5)
plt.plot(y_pos, ido['maximum'],'g+-', color = 'red', linewidth = 1, markersize = 6)
plt.plot(y_pos, ido['minimum'], 'g1-', color ='blue', linewidth = 1, markersize = 6)
plt.plot(y_pos, ido['atlag'], 'g+--', color = 'navy', linewidth = 1.5, markersize = 6)

plt.plot(y_pos, [ido.minimum.mean()]*num_days, color = 'blue', linewidth=.5, alpha = .5)
plt.plot(y_pos, [ido.maximum.mean()]*num_days, color = 'red', linewidth=.5, alpha = .5)

plt.legend(['Maximum ℃', 'Minimum ℃', 'Átlag ℃', 'Min. átlag', 'Max. átlag', 'Csapadék (mm)','Szél/10 km/h'])
plt.xticks(y_pos,ido['datum'], rotation=90)
plt.grid(alpha=.1, linestyle='-')
# plt.yticks(range(0,max_homerseklet + 2),range(0,max_homerseklet + 2))
plt.yticks(range(min_homerseklet, max_homerseklet))
plt.ylabel('Hőmérséklet / csapadék')
plt.title('Hőmérséklet és csapadék előrejelzés a következő '+str(num_days)+' napra: ' + telepules)
plt.savefig('ido.png')
plt.show()